# **1. Overview**
This project aims to predict the average price of golden delicious apples 

## **2. IMPORT DEPENDENCIES**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns

Matplotlib is building the font cache; this may take a moment.


## **3. LOAD DATASET**




In [29]:
train=pd.read_csv('data/df - train_set.csv')
test=pd.read_csv('data/df - test_set.csv')

## **4. DATA CLEANING**



### **4.1. SUMMARY OF DATAFRAME**
We obtained the column names and their data types

In [28]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64368 entries, 0 to 64375
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Province          64368 non-null  object 
 1   Container         64368 non-null  object 
 2   Size_Grade        64368 non-null  object 
 3   Weight_Kg         64368 non-null  float64
 4   Commodities       64368 non-null  object 
 5   Date              64368 non-null  object 
 6   Low_Price         64368 non-null  float64
 7   High_Price        64368 non-null  float64
 8   Sales_Total       64368 non-null  float64
 9   Total_Qty_Sold    64368 non-null  int64  
 10  Total_Kg_Sold     64368 non-null  float64
 11  Stock_On_Hand     64368 non-null  int64  
 12  avg_price_per_kg  64368 non-null  float64
dtypes: float64(6), int64(2), object(5)
memory usage: 6.9+ MB


The dataframe consists of 13 columns and 64368 observations.Furthermore it has 8 numerical columns, 4 categorical variables and 1 date column that is still in string format.

### **4.2. MISSING VALUES**
We checked if there are any missing values in each of the rows

In [24]:
train.isnull().sum()

Province            0
Container           0
Size_Grade          0
Weight_Kg           0
Commodities         0
Date                0
Low_Price           0
High_Price          0
Sales_Total         0
Total_Qty_Sold      0
Total_Kg_Sold       0
Stock_On_Hand       0
avg_price_per_kg    0
dtype: int64

The columns appear not to have any missing values

### **4.2 DATA ANOMALIES** 
We examined the data further and found that our response avg_price_per_kg column contained infinite and negative infinite values. We identified the relevant indices.

In [25]:
index_series=train[(train['avg_price_per_kg'] == math.inf) | (train['avg_price_per_kg'] == -math.inf)].index
index_series


Int64Index([1020, 9229, 21118, 25585, 39119, 40637, 46425, 60891], dtype='int64')

8 values out of a sample of over 60000 met our condition so we decided to discard them. With such a large sample it is safe to assume that the impact of removing a handful of observations is negligible

In [30]:
train.drop(index_series,inplace=True)

## **5. EXPLORATORY DATA ANALYSIS**



## **6. FEATURE SELECTION**

## **7. FEATURE ENGINEERING**

### **7.1. ENCODING CATEGORICAL FEATURES**

### **7.1.1. BINARY FEATURES**

### **7.1.2 ORDINAL FEATURES**

### **7.1.3. MULTI-CATEGORICAL FEATURES**

### **7.2. SPLITTING DATA**

### **7.3. FEATURE SCALING**

## **8. FITTING MODELS**